### 外部資料
**這裡的經緯度預設採用GoogleMaps**

In [2]:
import pandas as pd
import numpy as np
from glob import glob, iglob

df = pd.DataFrame()

# iglob使用較少memory
for csv in iglob('../0821/*_top500_clean.csv'):
    tmp_df = pd.read_csv(csv)
    df = df.append(tmp_df, ignore_index=True, sort=False)

In [3]:
len(df)

6195

### 內部資料(飯店經緯度)

In [4]:
df_gps = pd.read_csv("F:\\NCTU\\lab\\奧丁丁\\奧丁丁資料前處理\\OwlTing_整合資料\\csv\\data_gps.csv")

總共1500多家飯店

In [5]:
df_gps.shape

(1565, 5)

### 內部資料(飯店名稱)

In [6]:
df_name = pd.read_csv("F:\\NCTU\\lab\\奧丁丁\\奧丁丁資料前處理\\OwlTing_整合資料\\csv\\data.csv")
df_name = df_name[['hotel_id', 'name']].copy()

In [7]:
len(df_name.name.value_counts()) # 只有900家飯店知道名稱

889

只保留不重複的部分

In [8]:
df_name = df_name.drop_duplicates()

### 結合內部資料

In [9]:
df_gps = pd.merge(df_name, df_gps, left_on=['hotel_id'], right_on=['hotel_id'], how='outer')

緯度為null

In [7]:
df_gps.longitude.isna().sum()

232

In [3]:
df_gps.latitude.isna().sum()

73

經度為0

In [9]:
len(df_gps.loc[df_gps.latitude == 0])

364

In [11]:
len(df_gps.loc[df_gps.longitude == 0])

200

### 移除經緯度為0或是null的飯店

In [10]:
df_gps_clean = df_gps.dropna(subset=['latitude', 'longitude'])

In [11]:
# 僅刪除經緯度為0者
df_gps_clean = df_gps_clean[df_gps_clean.latitude!=0]
df_gps_clean = df_gps_clean[df_gps_clean.longitude!=0]

In [51]:
df_gps_clean.isna().sum()

hotel_id       0
name         372
country      196
city         339
latitude       0
longitude      0
dtype: int64

In [43]:
df_gps_clean.country.value_counts()

台灣          541
tw          182
TW          123
TAIWAN       42
MALAYSIA     14
Taiwan        7
Malaysia      7
日本            6
jp            3
台灣省           2
台湾            1
GREECE        1
my            1
中華民國          1
台灣TAIWAN      1
Name: country, dtype: int64

In [42]:
df_gps_clean.city.value_counts()

台南市        135
宜蘭縣         74
屏東縣         58
南投縣         45
花蓮          43
          ... 
瑞穗           1
Okinawa      1
Kythira      1
彰化           1
白沙鄉          1
Name: city, Length: 98, dtype: int64

**原始資料的經緯度就重複**

In [12]:
df.duplicated(subset=['lat', 'lng']).sum()

475

1. 使用內部資料全部的小數點

In [22]:
def get_first(x, length):
    return x[:length]

# 把經緯度成跟內部資料位數一樣
df['lat'] = df['lat'].astype('str').apply(get_first, args=(8,)).astype('float')
df['lng'] = df['lng'].astype('str').apply(get_first, args=(10,)).astype('float')

2. 內外部資料都只用小數點3位看能不能和

In [24]:
def get_first(x, length):
    return x[:length]

# 把經緯度成字串保留想要的位數
df['lat'] = df['lat'].astype('str').apply(get_first, args=(6,)).astype('float')
df['lng'] = df['lng'].astype('str').apply(get_first, args=(7,)).astype('float')
df_gps_clean['latitude'] = df_gps_clean['latitude'].astype('str').apply(get_first, args=(6,)).astype('float')
df_gps_clean['longitude'] = df_gps_clean['longitude'].astype('str').apply(get_first, args=(7,)).astype('float')

3. 小數點第四位後做四捨五入 **會完全和不了**

In [55]:
def get_round(x, length):
    return round(x, length)

df['lat'] = df['lat'].apply(get_round, args=(8,))
df['lng'] = df['lng'].apply(get_round, args=(9,))
df_gps_clean['latitude'] = df_gps_clean['latitude'].apply(get_round, args=(8,))
df_gps_clean['longitude'] = df_gps_clean['longitude'].apply(get_round, args=(9,))

4. 內外部資料都只用小數點4位

In [17]:
def get_first(x, length):
    return x[:length]

# 把經緯度成字串保留想要的位數
df['lat'] = df['lat'].astype('str').apply(get_first, args=(7,)).astype('float')
df['lng'] = df['lng'].astype('str').apply(get_first, args=(8,)).astype('float')
df_gps_clean['latitude'] = df_gps_clean['latitude'].astype('str').apply(get_first, args=(7,)).astype('float')
df_gps_clean['longitude'] = df_gps_clean['longitude'].astype('str').apply(get_first, args=(8,)).astype('float')

**經過處理後經緯度完全重複的**

In [18]:
df.duplicated(subset=['lat', 'lng']).sum()

555

In [19]:
df_gps_clean.duplicated(subset=['latitude', 'longitude']).sum()

31

經緯度同時重疊
> `keep=False` 會保留被重複的第一筆

In [58]:
df[df.duplicated(subset=['lat', 'lng'], keep=False)].sort_values(by=['lat']).sort_values(by=['lng'])

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count
764,麗馨精品商旅七賢館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.618,120.268,前金區市中一路229號,3.0,高雄,前金,"NT$817 - NT$2,169",4.0,18.0
574,奇異果快捷旅店-九如店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.618,120.268,三民區九如一路790號,3.0,高雄,三民,"NT$849 - NT$2,012",4.0,155.0
535,巴黎商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.625,120.280,新興區自立二路8號,3.0,高雄,新興,"NT$1,257 - NT$2,075",3.5,31.0
643,芳橙汽車旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.625,120.280,小港區中安路616號,3.0,高雄,小港,"NT$1,069 - NT$1,509",3.0,1.0
745,六合轉角737館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.622,120.283,南台路六合夜市 - 美麗島捷運站旁,2.0,高雄,新興,"NT$849 - NT$1,886",4.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...
1398,金色年代旅店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,25.121,121.861,板橋區重慶路66號12樓,3.0,新北,板橋,"NT$1,257 - NT$2,798",4.0,10.0
3661,大里驛青年旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.966,121.922,頭城鎮濱海路6段317-2號,2.0,宜蘭,頭城,"NT$1,980 - NT$3,018",5.0,16.0
3943,我家商務旅店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.966,121.922,羅東鎮中正路186之2號 3樓,2.5,宜蘭,羅東,"NT$1,100 - NT$3,018",0.0,0.0
1461,九份聽山,https://www.tripadvisor.com.tw/Hotel_Review-g1...,25.015,121.944,瑞芳區基山街219之4號,3.0,新北,瑞芳,"NT$943 - NT$2,703",3.0,11.0


In [20]:
df[df.duplicated(subset=['lat', 'lng'], keep=False)].groupby(['lat', 'lng']).agg('count')

,,title,uri,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count
lat,lng,,,,,,,,,
21.9250,120.8323,2,2,2,2,2,2,2,2,2
21.9289,120.8281,2,2,2,2,2,2,2,2,2
21.9334,120.8239,2,2,2,2,2,2,2,2,2
21.9335,120.8243,2,2,2,2,2,2,2,2,2
21.9337,120.8239,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...
25.1544,121.4613,2,2,2,2,2,2,2,2,2
25.1593,121.4558,2,2,2,2,2,2,2,2,2
25.1687,121.4453,2,2,2,2,2,2,2,2,2


### 嘗試能不能concat

內部資料的單一經緯度卻對應到多間外部資料的飯店

能和的飯店數目

In [22]:
len(pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']))

105

In [23]:
pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']).head(10)

,title,uri,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,lat,lng,hotel_id,name,country,city,latitude,longitude
0,羊兒煙囪,https://www.tripadvisor.com.tw/Hotel_Review-g1...,吉安鄉南海九街11號,0.0,花蓮,吉安,"NT$1,763 - NT$3,248",4.5,35.0,23.9746,121.6156,949,漫漫日出 Before Sunrise B&B,tw,花蓮市,23.9746,121.6156
1,檜木居民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,吉安鄉勝安村慈惠一街58號 23°58'34.2n 121°35'06.0e,3.5,花蓮,花蓮市,"NT$2,443 - NT$4,083",4.5,40.0,23.9972,121.5981,1321,花蓮鳶尾花民宿,台灣,花蓮市,23.9972,121.5981
2,薇琪小屋民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,花蓮市國興四街8號,3.5,花蓮,花蓮市,"NT$1,454 - NT$3,155",5.0,3.0,23.9746,121.6154,1346,海想念民宿,台灣,NaN,23.9746,121.6154
3,角舍背包客客棧,https://www.tripadvisor.com.tw/Hotel_Review-g1...,中山路743號,2.0,花蓮,花蓮市,"NT$773 - NT$2,474",4.5,3.0,23.9904,121.6048,1297,NaN,台灣,NaN,23.9904,121.6048
4,享海民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,中正路169巷1-7號,3.5,花蓮,花蓮市,-NT$1,5.0,14.0,23.9717,121.5976,962,享海民宿,台灣,花蓮,23.9717,121.5976
5,海芸民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,花蓮市北濱街43號,3.5,花蓮,花蓮市,"NT$1,763 - NT$3,155",4.5,5.0,23.9751,121.6148,1091,NaN,台灣,NaN,23.9751,121.6148
6,幸福加油站民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,信義街93號,3.0,花蓮,花蓮市,"NT$1,206 - NT$3,340",5.0,4.0,23.9762,121.6026,256,幸福加油站民宿,TW,花蓮市,23.9762,121.6026
7,幸福加油站民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,信義街93號,3.0,花蓮,花蓮市,"NT$1,206 - NT$3,340",5.0,4.0,23.9762,121.6026,1304,外館,台灣,花蓮縣,23.9762,121.6026
8,城市家園民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,花蓮市國威里大同街30-2號,3.0,花蓮,花蓮市,"NT$1,949 - NT$4,361",4.5,9.0,23.9801,121.6058,189,NaN,台灣,花蓮,23.9801,121.6058
9,美崙遊記民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,花蓮市中興路179號,3.5,花蓮,花蓮市,"NT$1,485 - NT$2,165",5.0,2.0,23.9692,121.6019,1087,法蘿拉民宿,台灣,NaN,23.9692,121.6019


檢查是否有同一間飯店被重複

In [24]:
pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']).title.value_counts()

高雄盧昂           3
海洋阿帕朵民宿        2
台中逢甲幻多奇青年旅棧    2
幸福加油站民宿        2
新世代精品商務旅店      2
              ..
城市家園民宿         1
欣欣渡假旅館         1
墾丁米拉貝拉         1
澤信旅店           1
墾丁 海灣羅曼史民宿     1
Name: title, Length: 99, dtype: int64

內部資料本來也有GPS重複的問題

In [25]:
df_tmp = pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude'])
df_tmp[df_tmp['title'].isin(df_tmp['title'][df_tmp['title'].duplicated()])]

,title,uri,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,lat,lng,hotel_id,name,country,city,latitude,longitude
6,幸福加油站民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,信義街93號,3.0,花蓮,花蓮市,"NT$1,206 - NT$3,340",5.0,4.0,23.9762,121.6026,256,幸福加油站民宿,TW,花蓮市,23.9762,121.6026
7,幸福加油站民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,信義街93號,3.0,花蓮,花蓮市,"NT$1,206 - NT$3,340",5.0,4.0,23.9762,121.6026,1304,外館,台灣,花蓮縣,23.9762,121.6026
15,高雄盧昂,https://www.tripadvisor.com.tw/Hotel_Review-g1...,鼓山區慶豐街98號,3.0,高雄,鼓山,"NT$912 - NT$3,018",0.0,0.0,22.6675,120.2946,1818,高雄盧昂,NaN,NaN,22.6675,120.2946
16,高雄盧昂,https://www.tripadvisor.com.tw/Hotel_Review-g1...,鼓山區慶豐街98號,3.0,高雄,鼓山,"NT$912 - NT$3,018",0.0,0.0,22.6675,120.2946,1872,NaN,NaN,NaN,22.6675,120.2946
17,高雄盧昂,https://www.tripadvisor.com.tw/Hotel_Review-g1...,鼓山區慶豐街98號,3.0,高雄,鼓山,"NT$912 - NT$3,018",0.0,0.0,22.6675,120.2946,1873,NaN,NaN,NaN,22.6675,120.2946
24,新世代精品商務旅店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,苓雅區三多四路63號14樓,2.5,高雄,苓雅,"NT$1,069 - NT$2,358",2.0,2.0,22.6117,120.3002,1480,85春天,tw,高雄市,22.6117,120.3002
25,新世代精品商務旅店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,苓雅區三多四路63號14樓,2.5,高雄,苓雅,"NT$1,069 - NT$2,358",2.0,2.0,22.6117,120.3002,2118,NaN,NaN,NaN,22.6117,120.3002
61,海洋阿帕朵民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,車城鄉後灣村後灣路183號,3.0,屏東,車城,"NT$2,382 - NT$2,969",5.0,1.0,22.0387,120.6940,717,住在 ZHUZAI,台灣,Pingtung County,22.0387,120.6940
62,海洋阿帕朵民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,車城鄉後灣村後灣路183號,3.0,屏東,車城,"NT$2,382 - NT$2,969",5.0,1.0,22.0387,120.6940,879,墾丁南法風情海景民宿,tw,車城鄉,22.0387,120.6940
71,台中逢甲幻多奇青年旅棧,https://www.tripadvisor.com.tw/Hotel_Review-g1...,文華路永新巷4號,2.0,台中,西屯,"NT$692 - NT$2,641",5.0,29.0,24.1767,120.6456,1629,台中逢甲幻多奇青年旅棧,TW,台中市,24.1767,120.6456


實際上和成功的飯店數 (單一經緯度先只保留一家，先不管內外部資料都有GPS重複的問題)

In [26]:
df_tmp.drop_duplicates(subset=['lat', 'lng'])

,title,uri,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,lat,lng,hotel_id,name,country,city,latitude,longitude
0,羊兒煙囪,https://www.tripadvisor.com.tw/Hotel_Review-g1...,吉安鄉南海九街11號,0.0,花蓮,吉安,"NT$1,763 - NT$3,248",4.5,35.0,23.9746,121.6156,949,漫漫日出 Before Sunrise B&B,tw,花蓮市,23.9746,121.6156
1,檜木居民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,吉安鄉勝安村慈惠一街58號 23°58'34.2n 121°35'06.0e,3.5,花蓮,花蓮市,"NT$2,443 - NT$4,083",4.5,40.0,23.9972,121.5981,1321,花蓮鳶尾花民宿,台灣,花蓮市,23.9972,121.5981
2,薇琪小屋民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,花蓮市國興四街8號,3.5,花蓮,花蓮市,"NT$1,454 - NT$3,155",5.0,3.0,23.9746,121.6154,1346,海想念民宿,台灣,NaN,23.9746,121.6154
3,角舍背包客客棧,https://www.tripadvisor.com.tw/Hotel_Review-g1...,中山路743號,2.0,花蓮,花蓮市,"NT$773 - NT$2,474",4.5,3.0,23.9904,121.6048,1297,NaN,台灣,NaN,23.9904,121.6048
4,享海民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,中正路169巷1-7號,3.5,花蓮,花蓮市,-NT$1,5.0,14.0,23.9717,121.5976,962,享海民宿,台灣,花蓮,23.9717,121.5976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,甜園玉光民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,礁溪鄉瑪僯路126-5號,3.0,宜蘭,礁溪,"NT$2,012 - NT$4,904",5.0,33.0,24.6708,121.5942,236,NaN,台灣,宜蘭縣,24.6708,121.5942
101,夏爾迦民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,五結鄉季新村季水路47號,3.5,宜蘭,五結,"NT$3,898 - NT$5,187",4.5,7.0,24.7236,121.7290,1246,Buluba民宿,TAIWAN,宜蘭,24.7236,121.7290
102,貴族大旅社,https://www.tripadvisor.com.tw/Hotel_Review-g1...,羅東鎮中正路186-3號4樓,3.0,宜蘭,羅東,"NT$1,006 - NT$1,918",4.0,1.0,24.6622,121.7379,1856,布克伍旅店 Book5,NaN,NaN,24.6622,121.7379
103,亞仕登旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,礁溪鄉忠孝路7號,3.5,宜蘭,礁溪,"NT$1,006 - NT$2,012",0.0,0.0,24.6729,121.8097,309,NaN,台灣,宜蘭縣,24.6729,121.8097
